In [ ]:
from dputils.scrape import Scraper, Tag

In [ ]:
url = 'https://www.flipkart.com/motorola-edge-50-pro-5g-68w-charger-black-beauty-256-gb/p/itmb3f6625c5c896?pid=MOBGXFXYDG8YRZU4&lid=LSTMOBGXFXYDG8YRZU4SAX9WW&marketplace=FLIPKART&store=tyy%2F4io&srno=b_1_1&otracker=browse&fm=organic&iid=ace38f80-151a-41b0-a079-b620b88b8ec9.MOBGXFXYDG8YRZU4.SEARCH&ppt=hp&ppn=homepage&ssid=vj43l0oe680000001721542596100'
s1 = Scraper(url)

In [ ]:
heading= Tag('h1', cls='_6EBuvT')
price = Tag(cls='Nx9bqj CxhGGd')
desc = Tag(cls='U+9u4y')
review = Tag(cls='RcXBOT')
results = s1.get_data_from_page(heading=heading, price=price, desc=desc, review=review)


multiple items from a single page

In [ ]:
items = Tag(cls='cPHDOP col-12-12')
url = "https://www.flipkart.com/tyy/4io/~cs-33irmtdz82/pr?sid=tyy%2C4io&collection-tab-name=Moto+Edge+50+pro+5g&param=279892&ctx=eyJjYXJkQ29udGV4dCI6eyJhdHRyaWJ1dGVzIjp7InZhbHVlQ2FsbG91dCI6eyJtdWx0aVZhbHVlZEF0dHJpYnV0ZSI6eyJrZXkiOiJ2YWx1ZUNhbGxvdXQiLCJpbmZlcmVuY2VUeXBlIjoiVkFMVUVfQ0FMTE9VVCIsInZhbHVlcyI6WyJGcm9tIOKCuTI3LDk5OSoiXSwidmFsdWVUeXBlIjoiTVVMVElfVkFMVUVEIn19LCJ0aXRsZSI6eyJtdWx0aVZhbHVlZEF0dHJpYnV0ZSI6eyJrZXkiOiJ0aXRsZSIsImluZmVyZW5jZVR5cGUiOiJUSVRMRSIsInZhbHVlcyI6WyJtb3Rvcm9sYSBlZGdlIDUwIFBybyJdLCJ2YWx1ZVR5cGUiOiJNVUxUSV9WQUxVRUQifX0sImhlcm9QaWQiOnsic2luZ2xlVmFsdWVBdHRyaWJ1dGUiOnsia2V5IjoiaGVyb1BpZCIsImluZmVyZW5jZVR5cGUiOiJQSUQiLCJ2YWx1ZSI6Ik1PQkdYRlhZTVRLQU1QVFMiLCJ2YWx1ZVR5cGUiOiJTSU5HTEVfVkFMVUVEIn19fX19"
s2 = Scraper(url)
results = s2.get_repeating_date_from_page(
    target = Tag('body'),
    items = items,
    title = Tag(cls='KzDlHZ'),
    price = Tag(cls='Nx9bqj _4b5DiR'),
    ratings = Tag('span', cls='Wphh3N')
)

In [ ]:
import pandas as pd
pd.DataFrame(results)

In [ ]:
items = Tag(cls='s-result-item')
title = Tag('span', cls='a-size-base-plus a-color-base a-text-normal')
url ='https://www.amazon.in/s?k=shoes&crid=2OAOAMZS2EVJY&sprefix=shoe%2Caps%2C306&ref=nb_sb_noss_1'
s3 = Scraper(url)
results = s3.get_repeating_date_from_page(
    target = Tag('body'),
    items = items,
    title = title,
    brand = Tag('h2', cls='a-size-mini s-line-clamp-1')
)
pd.DataFrame(results).dropna()

In [ ]:
start = 1
term = 'shoes'
all_data = []
while True:
    url = f"https://www.amazon.in/s?k={term}&page={start}&crid=2OAOAMZS2EVJY&qid=1721544045&sprefix=shoe%2Caps%2C306&ref=sr_pg_2"
    sc = Scraper(url)
    results = sc.get_repeating_date_from_page(
        target = Tag('body'),
        items = items,
        title = title,
        brand = Tag('h2', cls='a-size-mini s-line-clamp-1')
    )
    if len(results) == 0:
        break
    all_data.extend(results)
    start += 1

df = pd.DataFrame(all_data).dropna()
df

# sqlalchemy -> ORM (object relational mapping)

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.orm import Session, declarative_base
from datetime import datetime

Base = declarative_base()

class News(Base):
    __tablename__ = 'news'
    id = Column(Integer, primary_key=True)              # auto increment
    title = Column(String,  unique=True)
    content = Column(String)
    posted_by = Column(String)
    image = Column(String)
    created_at = Column(DateTime, default=datetime.now) # auto timestamp

engine = create_engine("sqlite:///news.sqlite", echo=True) # create a sqlite database
Base.metadata.create_all(engine)                           # create all tables

In [ ]:
from dputils.scrape import Scraper, Tag
url='https://www.ndtv.com/latest/page-2'
news_scraper = Scraper(url)
items = Tag(cls='news_Itm')
img_src = Tag('img', output='src')
title = Tag('h2', cls='newsHdng')
post = Tag('span', cls='posted-by')
content = Tag('p', cls='newsCont')
target = Tag('body')
out = news_scraper.get_repeating_date_from_page(
    target = target, items=items, 
    image=img_src, title=title, 
    post=post, content=content
)
db = Session(engine)
for entry in out:
    try:
        db.add(News(
        title = entry['title'],
        content = entry['content'],
        posted_by = entry['post'],
        image = entry['image']
        ))
    except Exception as e:
        print(e)

db.commit()
db.close()

In [13]:
import pandas as pd

news_df = pd.read_sql('news', engine)
news_df

2024-07-22 10:43:12,979 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-22 10:43:12,981 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("news")
2024-07-22 10:43:12,982 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-22 10:43:12,986 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-07-22 10:43:12,987 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-22 10:43:12,988 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-07-22 10:43:12,989 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-22 10:43:12,991 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("news")
2024-07-22 10:43:12,993 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-22 10:43:12,995 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in 

,id,title,content,posted_by,image,created_at
0,1,Gautam Gambhir Press Conference Live Updates: ...,Gautam Gambhir Press Conference Live: As the h...,"NDTV Sports Desk | Monday July 22, 2024 ...",https://c.ndtvimg.com/2024-07/2pvlbrmo_gambhir...,2024-07-22 10:35:02.605052
1,2,Byju's Founder Faces Reckoning As Company's Va...,"Byju Raveendran, the mathematics whiz who soar...","Reuters | Monday July 22, 2024, New Delhi ...",https://c.ndtvimg.com/2024-07/6h8u1do8_byju-ra...,2024-07-22 10:35:02.605052
2,3,South African Police Uncover Multimillion-Doll...,The South African Police Service (SAPS) and th...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tEdited by Anja...,https://c.ndtvimg.com/2024-07/ct8fl9vg_south-a...,2024-07-22 10:35:02.605052
3,4,None,None,None,None,2024-07-22 10:35:02.605052
4,5,Fresh Terror Attack After Army Chief's Visit T...,The Indian Army repulsed a major attack in Jam...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tWritten by Naz...,https://i.ndtvimg.com/i/2018-03/security-force...,2024-07-22 10:35:02.605052
5,6,Not Rohit Sharma Or Virat Kohli: Babar Azam's ...,Pakistan cricket team captain Babar Azam picke...,"NDTV Sports Desk | Monday July 22, 2024 ...",https://c.ndtvimg.com/2024-07/91k5o4i8_ab-de-v...,2024-07-22 10:35:02.605052
6,7,Gautam Gambhir Press Conference: 3 Big Questio...,India head coach Gautam Gambhir addresses medi...,"NDTV Sports Desk | Monday July 22, 2024 ...",https://c.ndtvimg.com/2024-07/67hfv6to_gautam-...,2024-07-22 10:35:02.605052
7,8,None,None,None,None,2024-07-22 10:35:02.605052
8,9,"Opinion | A Viksit Bharat Will Need Viksit, In...",Transforming both the ailing PSU and dormant v...,"Umakant Soni | Monday July 22, 2024 ...",https://c.ndtvimg.com/2024-07/kc1vpiog_umakant...,2024-07-22 10:35:02.605052
9,10,"Hours After Biden Drops Out, Kamala Harris Rec...",Democratic fundraising group ActBlue said Sund...,"Agence France-Presse | Monday July 22, 2024,...",https://c.ndtvimg.com/2024-07/5hctb05g_joe-bid...,2024-07-22 10:35:02.605052
